In [1]:
# =====================================================
# Cross-Domain Sentiment Classification (Books → Electronics)
# Using Spectral Feature Alignment (SFA)
# =====================================================

!pip install numpy scipy scikit-learn matplotlib nltk tqdm --quiet

import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import SpectralClustering
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.manifold import TSNE
from tqdm import tqdm
import nltk, re, urllib.request, zipfile, os

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# -----------------------------
# 1️⃣ Download & Prepare Dataset
# -----------------------------
# Amazon review dataset from Blitzer et al. (domain adaptation)
url = "https://www.cs.jhu.edu/~mdredze/datasets/sentiment/processed_acl.tar.gz"
if not os.path.exists("processed_acl.tar.gz"):
    urllib.request.urlretrieve(url, "processed_acl.tar.gz")

import tarfile
# Ensure extraction to the current directory or a specified path
extract_path = "."
if not os.path.exists("processed_acl"):
    with tarfile.open("processed_acl.tar.gz", "r:gz") as tar:
        tar.extractall(path=extract_path)

# Helper to load data from a domain folder
def load_domain(domain, label_limit=1000):
    # Corrected paths based on the observed extracted structure
    pos_files = [os.path.join(extract_path, f"processed_acl/{domain}/positive.review")]
    neg_files = [os.path.join(extract_path, f"processed_acl/{domain}/negative.review")]

    pos_texts = []
    for f_path in pos_files:
        if os.path.exists(f_path):
            with open(f_path, encoding="latin1") as f:
                pos_texts.extend(f.readlines()[:label_limit])

    neg_texts = []
    for f_path in neg_files:
        if os.path.exists(f_path):
            with open(f_path, encoding="latin1") as f:
                neg_texts.extend(f.readlines()[:label_limit])

    texts = [text.strip() for text in pos_texts + neg_texts]
    labels = np.array([1]*len(pos_texts) + [0]*len(neg_texts))
    return texts, labels

source_texts, source_labels = load_domain("books")
target_texts, target_labels = load_domain("electronics")

# -----------------------------
# 2️⃣ Text Cleaning & Vectorizing
# -----------------------------
def clean_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text.lower())
    return ' '.join([w for w in text.split() if w not in stop_words and len(w) > 2])

source_texts = [clean_text(t) for t in tqdm(source_texts)]
target_texts = [clean_text(t) for t in tqdm(target_texts)]

vectorizer = CountVectorizer(max_features=5000, binary=True)
X_source = vectorizer.fit_transform(source_texts)
X_target = vectorizer.transform(target_texts)
vocab = np.array(vectorizer.get_feature_names_out())

# -----------------------------
# 3️⃣ Identify Domain-Independent vs Domain-Specific Words
# -----------------------------
freq_source = np.array((X_source > 0).sum(axis=0)).ravel()
freq_target = np.array((X_target > 0).sum(axis=0)).ravel()

mask_independent = (freq_source > 30) & (freq_target > 30)
domain_independent = vocab[mask_independent]
domain_specific = vocab[~mask_independent]

print(f"Domain-independent words: {len(domain_independent)}")
print(f"Domain-specific words: {len(domain_specific)}")

# -----------------------------
# 4️⃣ Construct Co-occurrence Matrix (Bipartite Graph)
# -----------------------------
# For simplicity, co-occurrence = how often both appear in same document
M = np.zeros((len(domain_specific), len(domain_independent)))
# Create mappings for faster lookups
ds_vocab_map = {word: i for i, word in enumerate(domain_specific)}
di_vocab_map = {word: j for j, word in enumerate(domain_independent)}

# Combine source and target texts and their corresponding feature matrices for co-occurrence calculation
all_texts_combined = source_texts + target_texts
X_combined = vectorizer.transform(all_texts_combined)

for doc_idx in range(X_combined.shape[0]):
    doc_features = X_combined[doc_idx, :].nonzero()[1] # Get indices of words present in the document

    ds_present_indices = [ds_vocab_map[vocab[idx]] for idx in doc_features if vocab[idx] in ds_vocab_map]
    di_present_indices = [di_vocab_map[vocab[idx]] for idx in doc_features if vocab[idx] in di_vocab_map]

    for ds_map_idx in ds_present_indices:
        for di_map_idx in di_present_indices:
            M[ds_map_idx, di_map_idx] += 1


# -----------------------------
# 5️⃣ Spectral Clustering for Alignment
# -----------------------------
n_clusters = 100
# Adjusted affinity to handle potential zero rows in M
sc = SpectralClustering(n_clusters=n_clusters, affinity='cosine', assign_labels='kmeans', random_state=42)
# Handle cases where M might have zero rows (no domain-specific words co-occurring with DI words)
if M.shape[0] > 0 and np.sum(M) > 0:
    feature_clusters = sc.fit_predict(M)
else:
    # Assign all DS features to a single cluster if M is empty or all zeros
    feature_clusters = np.zeros(len(domain_specific), dtype=int)


# -----------------------------
# 6️⃣ Create Augmented Feature Representation
# -----------------------------
def augment_features(X, vocab, domain_specific, feature_clusters, n_clusters, vectorizer):
    X_new = np.zeros((X.shape[0], n_clusters))
    ds_word_to_cluster = {word: cluster for word, cluster in zip(domain_specific, feature_clusters)}

    for i, word in enumerate(vocab):
        if word in ds_word_to_cluster:
            cluster = ds_word_to_cluster[word]
            if word in vectorizer.vocabulary_:
                 idx = vectorizer.vocabulary_[word]
                 X_new[:, cluster] += X[:, idx].toarray().ravel()
    return np.hstack([X.toarray(), 0.6 * X_new])

X_source_aug = augment_features(X_source, vocab, domain_specific, feature_clusters, n_clusters, vectorizer)
X_target_aug = augment_features(X_target, vocab, domain_specific, feature_clusters, n_clusters, vectorizer)

# -----------------------------
# 7️⃣ Train on Source → Test on Target
# -----------------------------
clf = LogisticRegression(max_iter=1000)
clf.fit(X_source_aug, source_labels)
preds = clf.predict(X_target_aug)

print("\n📊 Classification Report (Books → Electronics):")
print(classification_report(target_labels, preds))

# -----------------------------
# 8️⃣ Inspect Learned Clusters
# -----------------------------
print("\n🔍 Example Word Clusters:")
# Ensure there are clusters before attempting to print
if n_clusters > 0 and len(domain_specific) > 0:
    for c in range(min(5, n_clusters)): # Print up to 5 clusters
        words_in_cluster = domain_specific[feature_clusters == c][:10]
        if len(words_in_cluster) > 0:
            print(f"Cluster {c}: {', '.join(words_in_cluster)}")
        else:
            print(f"Cluster {c}: No domain-specific words in this cluster.")
else:
    print("No clusters formed or no domain-specific words available to cluster.")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/tmp/ipython-input-1439152371.py:35: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_path)
100%|██████████| 2000/2000 [00:00<00:00, 2070.76it/s]


Domain-independent words: 363
Domain-specific words: 4637

📊 Classification Report (Books → Electronics):
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1000
           1       1.00      0.99      1.00      1000

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000


🔍 Example Word Clusters:
Cluster 0: admire, argued, australia, builds, cats, compromise, creatures, ethics, experimentation, experiments
Cluster 1: action, adams, additionally, admirable, admiration, adventure, affair, alert, alex, alternate
Cluster 2: absurd, accurately, acquire, applied, arguments, belief, believes, challenged, christians, claimed
Cluster 3: angst, bore, classes, clues, continually, decline, episode, frozen, lying, morality
Cluster 4: acquainted, afterwards, agreement, aids, applicable, assertion, benefits, compiled, condition, consideration